In [ ]:
import json
import re
from collections import Counter
from pathlib import Path

# ================================
# CONFIGURAÇÃO
# ================================
CAMINHO_ARQUIVO_JSON = r"C:\Users\Documents\Modelos\Perfil-Conciliação-Xgboost-GCP-V2\dados\anonymous_14052025\dados_processos_anonymous_14052025_V1_enriquecidos.json"
CAMINHO_RELATORIO_SAIDA = Path(CAMINHO_ARQUIVO_JSON).with_name("frequencia_tokens_artificiais.txt")

TOKENS_PADROES = [
    "NOME_PESSOA",
    "NOME_PESSOA_REP",
    "NUM",
    "NUM_REP",
    "VALOR_MONETARIO",
    "VALOR_MONETARIO_REP",
    "CIDADE_UF",
    "ENDERECO",
    "CEP",
    "CPF",
    "CNPJ"
]

# ================================
# FUNÇÕES
# ================================

def contar_tokens(texto: str, tokens_padroes):
    counter = Counter()
    for token in tokens_padroes:
        padrao = rf"\b{re.escape(token)}\b"
        ocorrencias = len(re.findall(padrao, texto))
        counter[token] += ocorrencias
    return counter

# ================================
# EXECUÇÃO
# ================================

def analisar_tokens_em_json(caminho_json, tokens_padroes):
    with open(caminho_json, "r", encoding="utf-8") as f:
        dados = json.load(f)

    contagem_total = Counter()
    n_docs = 0

    for item in dados:
        texto = item.get("inteiro_teor_limpo", "")
        if not texto:
            continue
        n_docs += 1
        contagem = contar_tokens(texto, tokens_padroes)
        contagem_total.update(contagem)

    # Relatório final
    relatorio = [f"Total de documentos analisados: {n_docs}\n"]
    for token in tokens_padroes:
        total = contagem_total[token]
        media = total / n_docs if n_docs else 0
        relatorio.append(f"{token:<25} → total: {total:>6} | média por doc: {media:.2f}")

    # Salva em arquivo
    with open(CAMINHO_RELATORIO_SAIDA, "w", encoding="utf-8") as f:
        f.write("\n".join(relatorio))

    # Também imprime no terminal
    print("\n".join(relatorio))
    print(f"\nRelatório salvo em: {CAMINHO_RELATORIO_SAIDA}")

# ================================
# EXECUTAR
# ================================
if __name__ == "__main__":
    analisar_tokens_em_json(CAMINHO_ARQUIVO_JSON, TOKENS_PADROES)